In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional

In [4]:
def calculate_cv2(file_path: str) -> Optional[pd.DataFrame]:
    """
    Tính CV² (Squared Coefficient of Variation) của khối lượng nhu cầu (demand volume)
    cho từng cặp (shop_id, item_id).

    CV² = (Độ lệch chuẩn của nhu cầu / Giá trị trung bình của nhu cầu)^2
    Chỉ xét những ngày có nhu cầu (sales > 0).
    """
    print("--- Bắt đầu tính toán CV² (Squared Coefficient of Variation) ---")
    
    try:
        df_sales = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp tại đường dẫn: {file_path}")
        return None

    # 1. Lọc nhu cầu thực tế (loại bỏ hàng trả lại hoặc giao dịch bằng 0)
    df_demand = df_sales[df_sales['item_cnt_day'] > 0].copy()

    # 2. Tổng hợp khối lượng nhu cầu hàng ngày (Demand Volume)
    df_daily_demand = df_demand.groupby(['shop_id', 'item_id', 'date'])['item_cnt_day'].sum().reset_index(name='demand_volume')

    # 3. Tính toán Thống kê (Mean và Standard Deviation) của khối lượng nhu cầu
    demand_stats = df_daily_demand.groupby(['shop_id', 'item_id'])['demand_volume'].agg(
        mean_demand=('mean'),
        std_dev_demand=('std')
    ).reset_index()

    # Xử lý trường hợp std_dev là NaN (chỉ có 1 lần bán hàng) bằng cách điền 0
    demand_stats['std_dev_demand'] = demand_stats['std_dev_demand'].fillna(0)

    # 4. Tính CV và CV²
    demand_stats['CV'] = demand_stats['std_dev_demand'] / demand_stats['mean_demand']
    demand_stats['CV2'] = demand_stats['CV'] ** 2

    # In kết quả thống kê
    print("\n--- Thống kê CV² (Squared Coefficient of Variation) ---")
    print(demand_stats['CV2'].describe())
    
    print("\n--- 5 cặp item/shop có CV² cao nhất (nhu cầu biến động mạnh) ---")
    top_cv2 = demand_stats.sort_values(by='CV2', ascending=False).head(5)
    
    # --- DÒNG ĐÃ SỬA LỖI ---
    # Thay thế .to_markdown(index=False) bằng print() thông thường
    print(top_cv2)

    return demand_stats

In [5]:
# --- Phần thực thi (tương đương với if __name__ == "__main__":) ---
    
# Giả sử tệp notebook (.ipynb) này được lưu trong thư mục 'src/'
current_dir = os.getcwd()

# Đường dẫn đúng đến tệp sales_train.csv
sales_path = os.path.join(current_dir, 'data', 'raw', 'sales_train.csv')

print(f"Đang phân tích dữ liệu từ: {sales_path}")

if not os.path.exists(sales_path):
    print(f"Lỗi: Không tìm thấy tệp {sales_path}.")
else:
    # Gọi hàm đã được định nghĩa ở ô trên và lưu kết quả
    cv2_results = calculate_cv2(sales_path)

Đang phân tích dữ liệu từ: c:\Users\user\OneDrive\Máy tính\doantt_KHDL\data_science_project\data_science_ecommerce\src\data\raw\sales_train.csv
--- Bắt đầu tính toán CV² (Squared Coefficient of Variation) ---

--- Thống kê CV² (Squared Coefficient of Variation) ---
count    424098.000000
mean          0.049385
std           0.330784
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          54.693475
Name: CV2, dtype: float64

--- 5 cặp item/shop có CV² cao nhất (nhu cầu biến động mạnh) ---
        shop_id  item_id  mean_demand  std_dev_demand        CV        CV2
64250        12    11373    14.144397      104.604937  7.395504  54.693475
61707        12     3731    13.924242       76.730760  5.510588  30.366578
178677       27     6111     3.603175       18.882950  5.240642  27.464334
153371       25     3732     3.957806       17.796350  4.496519  20.218685
280409       42    10210     6.013699       26.866177  4.467496  19.958524


In [6]:
# Bạn có thể xem lại kết quả (DataFrame) bất cứ lúc nào
if 'cv2_results' in locals():
    print("\nXem lại 5 dòng đầu của kết quả CV²:")
    print(cv2_results.head())


Xem lại 5 dòng đầu của kết quả CV²:
   shop_id  item_id  mean_demand  std_dev_demand        CV       CV2
0        0       30     3.444444        2.242271  0.650982  0.423777
1        0       31     1.571429        0.975900  0.621027  0.385675
2        0       32     1.454545        0.522233  0.359035  0.128906
3        0       33     1.000000        0.000000  0.000000  0.000000
4        0       35     1.250000        0.452267  0.361814  0.130909
